In [10]:
import sys
import pandas as pd
import numpy as np

In [2]:
og_input = pd.read_csv("~/NB-DEA/nb-deainput.csv", index_col=0)

In [4]:
samplenumA = 3
samplenumB = 3

threshold=10

In [77]:
samplestotal = samplenumA+samplenumB

counts = og_input.iloc[:, samplestotal: 2*samplestotal]

avglength = og_input.iloc[:, 2*samplestotal: 3*samplestotal]

counts['Total'] = counts.sum(axis=1)

rows_to_drop = counts[counts['Total'] < threshold].index.tolist()

counts.drop(rows_to_drop, inplace=True)
counts.drop('Total', axis=1, inplace=True)

avglength.drop(rows_to_drop, inplace=True)

In [8]:
avglength

,length.1,length.2,length.3,length.4,length.5,length.6
ENSMUSG00000000001,3213.00,3213.00,3213.00,3213.00,3213.00,3213.00
ENSMUSG00000000028,2094.00,1470.08,1957.50,1811.20,1867.80,2035.80
ENSMUSG00000000049,1141.00,1141.00,1141.00,1141.00,1141.00,1141.00
ENSMUSG00000000056,2548.30,4103.47,3224.95,4068.24,3289.98,3388.22
ENSMUSG00000000058,2289.75,1842.86,1471.19,1814.06,1738.50,1469.17
...,...,...,...,...,...,...
ENSMUSG00000099242,471.00,471.00,471.00,471.00,471.00,471.00
ENSMUSG00000099250,251.00,251.00,251.00,251.00,251.00,251.00
ENSMUSG00000099297,2381.00,2381.00,2381.00,2381.00,2381.00,2381.00
ENSMUSG00000099325,1140.00,1140.00,1140.00,1140.00,1140.00,1140.00


In [44]:
counts

,counts.1,counts.2,counts.3,counts.4,counts.5,counts.6
ENSMUSG00000000001,1212.0,1187.00,1154.00,1031.0,1274.00,848.00
ENSMUSG00000000028,15.0,32.00,10.00,16.0,11.00,13.00
ENSMUSG00000000049,18423.0,17917.00,15619.00,16395.0,16006.00,10709.00
ENSMUSG00000000056,515.0,593.00,606.00,801.0,733.00,553.00
ENSMUSG00000000058,71.0,60.00,44.00,45.0,46.00,41.00
...,...,...,...,...,...,...
ENSMUSG00000099242,12.0,6.00,7.00,8.0,12.00,11.00
ENSMUSG00000099250,120.0,293.00,134.28,44.0,315.50,90.50
ENSMUSG00000099297,11.0,21.00,16.00,13.0,25.00,5.00
ENSMUSG00000099325,3.0,1.50,2.00,5.5,3.50,0.50


In [13]:
means = counts.mean(axis=1)
variances = counts.var(axis=1)

## step1: normalization
want to account for
- sequencing depth
- gene length
- RNA composition

methods (for DE analysis)
- DESeq2's median of ratios
    - counts divided by sample-specific size factors determined by median ratio of gene counts relative to geometric mea n per gene
- EdgeR's trimmed mean of M values (TMM)
    - uses weighted trimmed mean of the log expression of ratios between samples

In [78]:
##using a geometric mean
counts2 = counts.copy()
arr_counts = counts2.to_numpy()
arr_counts[arr_counts == 0] = 1

pseudoref = np.prod(arr_counts, axis=1)**(1/samplestotal)
counts2["Psuedo-Reference"] = pseudoref

#ratios of each sample to reference
for i in range(samplestotal):
    colname = counts.columns[i]
    counts2['ratio.' + str(i+1)] = counts2[colname]/counts2["Psuedo-Reference"]
    #median of each column is that sample's sizefactor
    median = counts2['ratio.' + str(i+1)].median()
    
    #divide by size factor
    counts[colname] = counts[colname]/median

counts is now normalized!

## Differential Expression